# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115173e+02     1.541073e+00
 * time: 0.5685930252075195
     1     1.145653e+01     8.531373e-01
 * time: 1.5873689651489258
     2    -1.126371e+01     9.195932e-01
 * time: 2.076630115509033
     3    -3.348530e+01     7.078083e-01
 * time: 2.8074610233306885
     4    -4.686861e+01     5.363085e-01
 * time: 3.5236799716949463
     5    -5.676911e+01     1.947701e-01
 * time: 4.224524974822998
     6    -5.981909e+01     1.096191e-01
 * time: 4.714260101318359
     7    -6.095470e+01     4.779071e-02
 * time: 5.18532395362854
     8    -6.137520e+01     5.232300e-02
 * time: 5.659131050109863
     9    -6.163378e+01     3.606315e-02
 * time: 6.139017105102539
    10    -6.184583e+01     2.576030e-02
 * time: 6.62230110168457
    11    -6.199628e+01     2.074801e-02
 * time: 7.096053123474121
    12    -6.206197e+01     1.504268e-02
 * time: 7.570094108581543
    13    -6.212302e+01     1.340728e-02
 * time: 8.046975135803223
    1

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671067
    AtomicLocal         -18.8557662
    AtomicNonlocal      14.8522632
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336820

    total               -62.261666458281
